In [3]:
import requests
import json
import re
import networkx as nx
%matplotlib inline
import pandas as pd

In [4]:

def req2():
    requestUrl = "https://api.nytimes.com/svc/archive/v1/2020/11.json?api-key=A9IKQgYHxfUzG2FCHgpzO4kBypcsV67w"
    #requestHeaders = {"Accept": "application/json"} headers=requestHeaders
    #query = {"year":2020, "month":2}
    
    response = requests.get(requestUrl)

    #print(response.text)
    response2 = response.json()
    return response2
    

response = req2()

In [5]:
z = response['response']['docs']

In [6]:
main_dict = {}

for x in z:
    type_of_material = x['type_of_material']
    j1 = x['byline']['original']
    k2 = j1.replace('By','')
    lq = k2.replace('and',',').split(',')
    
    #print(f'Type of material: {type_of_material}')
    #print('Author(s)')
    
    for x in lq:
        if x != '' and type_of_material != '':
            main_dict.setdefault(type_of_material, []).append(x)

#for k, v in main_dict.items():
    #print(k, v)
    #print(k)


In [7]:
author_name_map = {}
author_genre_map = {}

In [8]:
#def att(x):
for x in z:
    type_of_material = x['type_of_material']
    j1 = x['byline']['original']
    h = j1.replace('By ', '')
    s = h.replace(', ', ',')
    t = s.replace(' and ', ',').split(',')
     
    # For each author, add the genre to that author's list
    for r in t:
        if r != '':
            author_name_map[r] = r
        
        
            this_authors_genres = author_genre_map.get(r, {})
        #print(this_authors_genres)
            
            # Increment the count of genres for this actor
        #for g in type_of_material:
            if type_of_material != '':
                this_authors_genres[type_of_material] = this_authors_genres.get(type_of_material, 0) + 1
        #print(this_authors_genres)
        
         # Update the map
                author_genre_map[r] = this_authors_genres


In [103]:
#for k, v in author_genre_map.items():
    #print(f'key:{k}, Value:{v}')
    #print(k, v)

In [84]:
author_genre_map['Nate Cohn']

{'News': 12, 'Interactive Feature': 1}

In [111]:
# Get all actors as an index for a dataframe
index = author_genre_map.keys()

# Get the genre-counts for each actor in the index
rows = [author_genre_map[k] for k in index]

# Create the data frame from these rows, with the actors as index
df = pd.DataFrame(rows, index=index)

# Fill NAs with zero, as NA means the actor has not starred in that genre
df = df.fillna(0)

df.head(5)

,News,Interactive Feature,Review,News Analysis,Video,An Appraisal,briefing,Editorial,Obituary (Obit),Op-Ed,List,Letter,Biography
Nick Corasaniti,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Keith Collins,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Trip Gabriel,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stephanie Saul,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jennifer Medina,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [107]:
#Setting the actor we will be comparing to
target_author_id = 'Nate Cohn'

In [108]:
#Gathering the genres for that actor
target_author_ratings = df.loc[target_author_id]
#print(target_author_ratings)

In [109]:
#Generating distances from that actor to all the others
distances = scipy.spatial.distance.cdist(df, [target_author_ratings], metric="cosine")[:,0]

query_distances = list(zip(df.index, distances))
#print(query_distances)

In [110]:
#Printing the top ten most similar actors to our target
similar_list = []
for similar_actor_id, similar_genre_score in sorted(query_distances, key=lambda x: x[1], reverse=False)[:11]:
    similar_actor = author_genre_map[similar_actor_id]
    #print(similar_actor_id, author_name_map[similar_actor_id], similar_genre_score)
    similar_list.append({'Authors': author_name_map[similar_actor_id], 'Similar Score': similar_genre_score})
similar_df = pd.DataFrame(similar_list)
similar_df = similar_df.tail(-1).set_index('Authors')
similar_df

,Similar Score
Authors,
Emma G. Fitzsimmons,0.000020
Jeremy Engle,0.000020
Bill Pennington,0.000028
Trip Gabriel,0.000070
Giulia McDonnell Nieto del Rio,0.000137
Tiffany Hsu,0.000137
Alan Blinder,0.000297
Stephanie Saul,0.000550
Michael Cooper,0.000849
